# Accessing SCKAN from python
This notebook contains useful links for working with SCKAN from python.

Examples below show how to work with SCKAN directly from github using `neurondm`,  
as well how to query SCKAN using SPARQL.

## Useful links
### General introduction to `neurondm`
https://github.com/tgbugs/pyontutils/blob/master/neurondm/docs/NeuronLangExample.ipynb

### Retrieving neurons from git with `neurondm` extended example
https://github.com/tgbugs/pyontutils/blob/master/neurondm/docs/composer.py

### Example queries
https://github.com/SciCrunch/sparc-curation/blob/master/docs/queries.org

### General notebook setup
https://github.com/tgbugs/pyontutils/blob/master/neurondm/docs/neurons_notebook.md

### Docker setup
https://github.com/SciCrunch/sparc-curation/blob/master/docs/sckan/README.org

### Python setup
If you already have a python environment that can run notebooks run the following.

```bash
pip install neurondm
```

# Python

In [ ]:
import os
import rdflib
from pyontutils.core import OntGraph, OntResIri, OntResPath
from pyontutils.namespaces import rdfs, ilxtr
from neurondm.core import Config, graphBase, log
from neurondm.core import OntTerm, OntId, RDFL
from neurondm import orders
from neurondm import lang

config = Config('random-merge', ttl_export_dir="NIF-Ontology")
g = OntGraph()  # load and query graph

# remove scigraph and interlex calls
graphBase._sgv = None
del graphBase._sgv
if len(OntTerm.query._services) > 1:
    # backup services and avoid issues on rerun
    _old_query_services = OntTerm.query._services
    _noloc_query_services = _old_query_services[1:]

OntTerm.query._services = (RDFL(g, OntId),)

# base paths to ontology files
gen_neurons_path = 'ttl/'
suffix = '.ttl'
orr = 'https://raw.githubusercontent.com/SciCrunch/NIF-Ontology/'
branch = 'master/'
remote_base = orr + branch + gen_neurons_path

# full imports
for f in ('NIF-Organism',):
    ori = OntResIri(remote_base + f + suffix)
    [g.add(t) for t in ori.graph]

config.load_existing(g)
neurons = config.neurons()  # scigraph required here if deps not removed above

In [ ]:
print(neurons[0])

# SPARQL

You can query SCKAN using SPARQL via https://blazegraph.scicrunch.io/blazegraph/sparql.

Use an HTTP GET request with a urlencoded `?query={urlenc-query}` parameter.

Here is an example using `curl`. Another example using python is shown below.

```bash
curl \
-H "Accept: text/csv" \
--data-urlencode "query=SELECT ?p ?o WHERE { build:prov ?p ?o }" \
https://blazegraph.scicrunch.io/blazegraph/sparql | \
sed -e 's,http://uri.interlex.org/tgbugs/uris/readable/build/,build:,g' \
    -e 's,http://www.w3.org/1999/02/22-rdf-syntax-ns#,rdf:,g' \
    -e 's,http://www.w3.org/2000/01/rdf-schema#,rdfs:,g' \
    -e 's,http://www.w3.org/2002/07/owl#,owl:,g'
```
```csv
p,o
build:date,2023-08-04
build:datetime,"2023-08-04T00:58:09,027766Z"
build:epoch,1691110689
build:id,build:id/42ceb37f-a2e0-4222-abcf-0b7f5c2916dd
build:metaVersion,0
build:time,00:58:09.027766
build:type,build:Blazegraph
rdf:type,owl:NamedIndividual
rdf:type,build:Record
rdfs:label,graph build and load provenance record
```

In [ ]:
import io
import csv
import requests
from urllib.parse import quote as url_quote

blazegraph_endpoint = 'https://blazegraph.scicrunch.io/blazegraph/sparql'


def procq(res):
    _, (str_count,) = res
    return int(str_count)


def query(query, *, endpoint=blazegraph_endpoint, **kwargs):
    qq = url_quote(query, safe='')
    url = f'{endpoint}?query={qq}'
    headers = {'Accept': 'text/csv'}
    resp = requests.get(url, headers=headers)
    return list(csv.reader(io.StringIO(resp.text)))

In [ ]:
# copy and paste (and edit as needed) queries to retrieve data

# from https://github.com/SciCrunch/sparc-curation/blob/master/docs/queries.org#npo-partial-orders
# npo partial orders query for neurons returns an adjacency list of region layer pairs
example_query = """
select distinct
?s
?region_1 ?layer_1
?region_2 ?layer_2
where {
  # common
  ?s ilxtr:neuronPartialOrder ?o .
  ?o (rdf:rest|rdf:first)* ?r1 .
  ?r1 (rdf:rest|rdf:first)* ?r2 .

  { # region       region
    ?region_1 a owl:Class .
    ?region_2 a owl:Class .

    ?r1 rdf:first ?region_1 .
    ?r2 rdf:first ?region_2 .
    filter (?mediator = ?r1)  # draw only from the same partial order
    ?mediator rdf:first ?region_1 .  # car
    ?mediator rdf:rest+/rdf:first/rdf:first ?region_2 .  # caadr
  }
  union
  { # region layer region
    ?region_1 a owl:Class .
    ?layer_1 a owl:Class .
    ?region_2 a owl:Class .

    ?r1 rdf:first [ ?region_1 ?layer_1 ] .
    ?r2 rdf:first ?region_2 .
    filter (?mediator = ?r1)  # draw only from the same partial order
    ?mediator rdf:first [ ?region_1 ?layer_1 ] .  # car
    ?mediator rdf:rest+/rdf:first/rdf:first ?region_2 .  # caadr
  }
  union
  { # region       region layer
    ?region_1 a owl:Class .
    ?region_2 a owl:Class .
    ?layer_2 a owl:Class .

    ?r1 rdf:first ?region_1 .
    ?r2 rdf:first [ ?region_2 ?layer_2 ] .
    filter (?mediator = ?r1)  # draw only from the same partial order
    ?mediator rdf:first ?region_1 .  # car
    ?mediator rdf:rest+/rdf:first/rdf:first [ ?region_2 ?layer_2 ] .  # caadr
  }
  union
  { # region layer region layer
    ?region_1 a owl:Class .
    ?layer_1 a owl:Class .
    ?region_2 a owl:Class .
    ?layer_2 a owl:Class .

    ?r1 rdf:first [ ?region_1 ?layer_1 ] .
    ?r2 rdf:first [ ?region_2 ?layer_2 ] .
    filter (?mediator = ?r1)  # draw only from the same partial order
    ?mediator rdf:first [ ?region_1 ?layer_1 ] .  # car
    ?mediator rdf:rest+/rdf:first/rdf:first [ ?region_2 ?layer_2 ] .  # caadr
  }
} order by ?s ?region_1 ?layer_1 ?region_2 ?layer_2
"""

result = query(example_query)

In [ ]:
result[:2]